## SRJ

## qubit adapt

In [3]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 2.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886 
r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    print('singles', singles)
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                S[i,j] = left_op.dot(right_op).real
     
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
          
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state


Theta is 0.7853981633974483


In [4]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state = adaptvqe(adapt_it=2, e_th=1e-12)

HF state is -2.07524282897348
The operator pool length is 26
singles [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
The adapt iteration now is 1
The measurement of new state is {np.str_('11110000'): np.int64(1000)}
The operator check for this iteration 1 is []


c:\Users\poyy468\AppData\Local\miniconda3\envs\cwq\Lib\site-packages\pennylane_lightning\core\_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.27478610936536707 for operator a⁺(2) a⁺(3) a(4) a(5)
Highest gradient excitation is [2, 3, 4, 5]
ash_excitation is [[2, 3, 4, 5]]
The length of ash_excitation before generating matrix is 1
0: ─╭|Ψ⟩───────────┤  State
1: ─├|Ψ⟩───────────┤  State
2: ─├|Ψ⟩─╭G²(0.79)─┤  State
3: ─├|Ψ⟩─├G²(0.79)─┤  State
4: ─├|Ψ⟩─├G²(0.79)─┤  State
5: ─├|Ψ⟩─╰G²(0.79)─┤  State
6: ─├|Ψ⟩───────────┤  State
7: ─╰|Ψ⟩───────────┤  State
State count after Generating function is {np.str_('11001100'): np.int64(143), np.str_('11110000'): np.int64(857)}
Skipping individual excitation state for j < 2
After adding the states the adapt iteration is 1 and no of states 2
The value of i and j is 0 0
The value of i and j is 0 1
The value of i and j is 1 0
The value of i and j is 1 1
S matrix with 1 and 1 is [[1.        +0.j 0.92387953-0.j]
 [0.92387953+0.j 1.        +0.j]]
Hamiltonian Matrix with 1 1 is [[-2.07524283+0.j -1.96985242-0.j]
 [-1.96985242+0.j -2.06918289+0.j]]
The shape of S matri

In [ ]:
import pennylane as qml
import numpy as np
exc = []
electrons = 4
orbitals = 8
# This below command will generate the excitations. 
singles, doubles = qml.qchem.excitations(electrons, orbitals)

# Fermionic operator is created here
op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
#print(op1)
op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
#print(op2[-1])
operator_pool = op1 + op2

#Convert to qubit operator using JW transformation
for i in operator_pool:
        print('The current excitation operator is', i) #Current excitation operator - fermionic one
        #print('The value of  operator check is', operator_check)  #Max excitation operator
        w = qml.fermi.jordan_wigner(i)
        print('Value after JW', w) 
        exc.append(w) # This command stores all the operators in Pauli representation
#print('All exc', exc[0])


The current excitation operator is a⁺(0) a(4)
Value after JW -0.25j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + (0.25+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (0.25+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.25j * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
The current excitation operator is a⁺(0) a(6)
Value after JW -0.25j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (0.25+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (0.25+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.25j * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
The current excitation operator is a⁺(1) a(5)
Value after JW -0.25j * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + (0.25+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5)) + (0.25+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5)) + 0.25j * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
The current excitation operator is a⁺(1) a(7)
Value after JW -0.25j * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7)) + (0.25+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7)) + (0.25+0j) * (X(1

In [18]:
import pennylane as qml
from pennylane.pauli import pauli_sentence
import numpy as np
import random

exc_paulis = []
electrons = 4
orbitals = 8

singles, doubles = qml.qchem.excitations(electrons, orbitals)
op1 = [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
op2 = [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"}) for x in doubles]
operator_pool = op1 + op2

for i, ferm_op in enumerate(operator_pool):
    print(f"\nFermionic excitation operator {i}: {ferm_op}") # Fermionic exc.
    pauli_sum = qml.fermi.jordan_wigner(ferm_op) #JW transfomration
    pauli_sentence_obj = pauli_sentence(pauli_sum) # Converting to pauli sentence than sum
    print('PauliSentence after JW:', pauli_sentence_obj)
    for pauli_word, coeff in pauli_sentence_obj.items():
        op_dict = dict(pauli_word.items())
        print(op_dict)
        label = ''
        for wire in range(orbitals):
            label += op_dict.get(wire, 'I')
        rand_coeff = -1 * random.random()
        exc_paulis.append((rand_coeff, label, coeff))
        print(f"Pauli string: {label}, JW coeff: {coeff}, random coeff: {rand_coeff}")



Fermionic excitation operator 0: a⁺(0) a(4)
PauliSentence after JW: -0.25j * Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)
+ (0.25+0j) * Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)
+ (0.25+0j) * X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)
+ 0.25j * X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)
{0: 'Y', 1: 'Z', 2: 'Z', 3: 'Z', 4: 'X'}
Pauli string: YZZZXIII, JW coeff: -0.25j, random coeff: -0.27795235124670215
{0: 'Y', 1: 'Z', 2: 'Z', 3: 'Z', 4: 'Y'}
Pauli string: YZZZYIII, JW coeff: (0.25+0j), random coeff: -0.19772214015940748
{0: 'X', 1: 'Z', 2: 'Z', 3: 'Z', 4: 'X'}
Pauli string: XZZZXIII, JW coeff: (0.25+0j), random coeff: -0.8844856632415965
{0: 'X', 1: 'Z', 2: 'Z', 3: 'Z', 4: 'Y'}
Pauli string: XZZZYIII, JW coeff: 0.25j, random coeff: -0.3518945129622769

Fermionic excitation operator 1: a⁺(0) a(6)
PauliSentence after JW: -0.25j * Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)
+ (0.25+0j) * Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)
+ (0.25+0j) * X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)
+ 0.25j * X(0) @ Z(1) @ Z(2) @